In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn as sk

import matplotlib.pyplot as plt
from datetime import datetime

import sklearn
print("start")

f=open(r'.\交通银行601328.csv')
d=pd.read_csv(f)

d= d.sort_index(ascending=False)  #按照index降序排列
#print(d)
#print("///////////////////////////////")
d=d.reset_index(drop=True)
#print(d)
d_row=d.shape[0]
d['label']=1  #我们认为1代表涨，0代表不涨不跌，-1代表跌
#print(d)

#print(d.iloc[1,0])
#print(d.iloc[1,2])
#print(d.iloc[0,2])
#  -------------------建立标签----------------
for i in range(1,d_row):   
    if(d.iloc[i,2]>=d.iloc[i-1,2]):
        d.iloc[i,6]=1       #涨，记作1，编码形式为[1,0]
    else:
        d.iloc[i,6]=0

d=d.drop([0])
d=d.reset_index(drop=True)
d_row=d.shape[0]  #删除第一行以后，现在是2418行了

#d=d.iloc[:,:].values
#print(d)

d=d.iloc[:,:].values

before=10
after=1

train_x=[]
train_y=[]
input_list=[]
label_list=[]
label_date_list=[] #带有date的label

size=d_row-1-(before+after-1)+1
for i in range(d_row-1-(before+after-1)+1):
    x1=d[i:i+before,1:5]  #第0列是日期，取1到4列，分别是open，close，high，low
    x1=x1.tolist()
    input_list.append(x1)
    
    y1=d[i+before:i+before+after,6:7]  #出错点，只取第6号列，label
    y1=y1.tolist() #出错点,y1此时的形状是(after,1),我们想要的是(1,after)，要进行转换。其实也可不用，之后有reshape(-1,n_classes)语句会转换
    
    if(y1[0][0]==1):    #此处手动转换标签！
        y1=np.array([1,0]).reshape(1,2)      #[1,0]代表涨
    else:
        y1=np.array([0,1]).reshape(1,2)
        
    label_list.append(y1)
    
   # y2=d[i+before:i+before+after,0:5]
   # y2=y2.tolist()
   # y2=np.array(y2).reshape(1,after)
   # label_date_list.append(y2)

#print(len(input_list))
input_list=np.array(input_list)
#print(input_list.shape)  #(size,before,4) ,4是指提出了x的4个属性
label_list=np.array(label_list)

#    ------------划分训练集，测试集-------------
train_proportion=0.7
end=int(size*train_proportion)
#print("end:",end)

train_x=input_list[0:end,:,:]
#print(train_x.shape)  #(1447,5,4)
train_y=label_list[0:end,:,:]

test_x=input_list[end:size,:,:]
test_y=label_list[end:size,:,:]

#print("test_y.shape:",test_y.shape)  #（724,1,2）
#print("test_y:",test_y)

ups=0
downs=0
for i in range(test_y.shape[0]):
    if(test_y[i][0][0]==1):
        ups+=1
    else:
        downs+=1
print("测试集标签总数：",test_y.shape[0])
print("测试集上涨数及比例：",ups,ups/test_y.shape[0])
print("测试集下跌数及比例：",downs,downs/test_y.shape[0])

batch_index=[]  #对于输入，按照批次喂入网络
batch_size=60

for i in range(train_x.shape[0]):
    if(i%batch_size==0 or i==train_x.shape[0]-1):
        batch_index.append(i)           #建立每次喂入的索引号，比如0,60,120,125

print("len(batch_size):",len(batch_index))
        
input_size=4  #input_size=x的属性数量
time_steps=before  #
num_units=64 #num_units
n_classes=2   # 涨，跌

x=tf.placeholder(tf.float32,[None,time_steps,input_size])
y=tf.placeholder(tf.float32,[None,n_classes])   #y是label，正确答案

w=tf.Variable(tf.truncated_normal([num_units,n_classes],stddev=0.1))
b=tf.Variable(tf.constant(0.1,shape=[n_classes]))

def lstm(X,w,b):
    #inputs=tf.reshape(X,[-1,time_steps,input_size])
    inputs=X
    lstm_cell=tf.contrib.rnn.BasicLSTMCell(num_units)
    outputs,final_state=tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32) #tf.nn.dynamic_rnn最好参见tensorflow官网解释
    #final_state[0]=cell state
    #final_state[1]=hidden state
    #results=tf.nn.softmax(tf.matmul(final_state[1],w)+b)
    results=tf.nn.tanh(tf.matmul(final_state[1],w)+b)
    return results
    
prediction=lstm(x,w,b)
y_p=tf.argmax(prediction,1)

cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
#loss=tf.reduce_mean(tf.square(y-prediction))
train_step=tf.train.AdamOptimizer(1e-3).minimize(cross_entropy)   #hinton建议设置为1e-3，代表初始学习率

correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))      

"""
test = np.array([[1, 2, 3], [2, 3, 4], [5, 4, 3], [8, 7, 2]])
np.argmax(test, 0)　　　＃输出：array([3, 3, 1]
np.argmax(test, 1)　　　＃输出：array([2, 2, 0, 0]

原文：https://blog.csdn.net/qq575379110/article/details/70538051 

"""
with tf.Session() as sess:
    init_op=tf.global_variables_initializer()
    sess.run(init_op)
    for epoch in range(10):
        for step in range(len(batch_index)-1):
            y_trans=(train_y[batch_index[step]:batch_index[step+1]]).reshape(-1,n_classes)  #转换之前y的形状是,(-1,1,3)
            sess.run(train_step,feed_dict={x:train_x[batch_index[step]:batch_index[step+1]],y:y_trans})
            
            
            
            #loss=sess.run(cross_entropy,feed_dict={x:train_x[batch_index[step]:batch_index[step+1]],y:y_trans})
            #print(str(epoch)+" epoch,the loss is:"+str(loss))
        c_e=sess.run(cross_entropy,feed_dict={x:train_x[batch_index[step]:batch_index[step+1]],y:y_trans})
       # print(str(epoch)+" times,cross_entropy:"+str(c_e))    
        
        acc=sess.run(accuracy,feed_dict={x:test_x,y:test_y.reshape(-1,n_classes)})
        y_pred=sess.run(y_p,feed_dict={x:test_x,y:test_y.reshape(-1,n_classes)})
        
        pred=sess.run(prediction,feed_dict={x:test_x,y:test_y.reshape(-1,n_classes)})
        print("prediction:",pred)
        
        
        train_acc=sess.run(accuracy,feed_dict={x:train_x,y:train_y.reshape(-1,n_classes)})
        #print(str(epoch)+" times,train accuracy:"+str(train_acc))
        
        print(str(epoch)+" times,test accuracy:"+str(acc))
        
        y_true = np.argmax(test_y.reshape(-1,n_classes),1)
        #print("y_true:",y_true)
        print("Precision", sk.metrics.precision_score(y_true, y_pred))
        print( "Recall", sk.metrics.recall_score(y_true, y_pred))
        print( "f1_score", sk.metrics.f1_score(y_true, y_pred))
        print( "confusion_matrix")
        print(sk.metrics.confusion_matrix(y_true, y_pred))
        #fpr, tpr, tresholds = sk.metrics.roc_curve(y_true, y_pred)

#作者：wipen
#链接：https://www.jianshu.com/p/fb9e9f413791

        #print("y_predict:",y_predict)

print("over")

start
测试集标签总数： 723
测试集上涨数及比例： 390 0.5394190871369294
测试集下跌数及比例： 333 0.4605809128630705
len(batch_size): 30
Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

prediction: [[0.23457375 0.3708278 ]
 [0.22283904 0.36668834]
 [0.23207957 0.37300998]
 ...
 [0.21184205 0.37093848]
 [0.2117063  0.3704624 ]
 [0.21280953 0.37169537]]
0 times,test accuracy:0.46058092
Precision 0.4605809128630705
Recall 1.0
f1_score 0.6306818181818182
confusion_matrix
[[  0 390]
 [  0 333]]
prediction: [[0.34661287 0.3786353 ]
 [0.33553103 0.37433216]
 [0.3435754  0.38002512]
 ...
 [0.32474783 0.3793572 ]
 [0.3247549  0.378995  ]
 [0.3258441  0.3803

prediction: [[0.73812044 0.76944524]
 [0.7281158  0.76856846]
 [0.73616385 0.7690513 ]
 ...
 [0.7139299  0.76410085]
 [0.71389765 0.7640344 ]
 [0.715013   0.76459634]]
23 times,test accuracy:0.48962656
Precision 0.47068403908794787
Recall 0.8678678678678678
f1_score 0.6103484688489969
confusion_matrix
[[ 65 325]
 [ 44 289]]
prediction: [[0.6347797  0.6825589 ]
 [0.62038034 0.68104494]
 [0.6325275  0.6821463 ]
 ...
 [0.60282975 0.6761986 ]
 [0.60264283 0.67599803]
 [0.604126   0.67664933]]
24 times,test accuracy:0.4923928
Precision 0.4729299363057325
Recall 0.8918918918918919
f1_score 0.6181061394380853
confusion_matrix
[[ 59 331]
 [ 36 297]]
prediction: [[0.7522357  0.753767  ]
 [0.73946744 0.75062686]
 [0.74997175 0.7529174 ]
 ...
 [0.70712847 0.7283216 ]
 [0.707787   0.7288891 ]
 [0.70979595 0.73023623]]
25 times,test accuracy:0.54771787
Precision 0.5086206896551724
Recall 0.5315315315315315
f1_score 0.5198237885462554
confusion_matrix
[[219 171]
 [156 177]]
prediction: [[0.6613523  

prediction: [[0.72385335 0.6026299 ]
 [0.69470125 0.6023386 ]
 [0.718266   0.6016865 ]
 ...
 [0.6319007  0.5676108 ]
 [0.6321869  0.56817603]
 [0.63567126 0.5692925 ]]
49 times,test accuracy:0.54080224
Precision 0.5111111111111111
Recall 0.06906906906906907
f1_score 0.1216931216931217
confusion_matrix
[[368  22]
 [310  23]]
prediction: [[0.5330639  0.54460967]
 [0.48803818 0.54525286]
 [0.5215778  0.5417779 ]
 ...
 [0.43013704 0.5298286 ]
 [0.42997307 0.52992254]
 [0.43418375 0.53050333]]
50 times,test accuracy:0.5546335
Precision 0.516320474777448
Recall 0.5225225225225225
f1_score 0.5194029850746269
confusion_matrix
[[227 163]
 [159 174]]
prediction: [[0.5545044  0.5798312 ]
 [0.5108327  0.5800465 ]
 [0.5442991  0.57806844]
 ...
 [0.43531927 0.5525004 ]
 [0.43549022 0.55288523]
 [0.44004628 0.5537539 ]]
51 times,test accuracy:0.54771787
Precision 0.5082872928176796
Recall 0.5525525525525525
f1_score 0.5294964028776978
confusion_matrix
[[212 178]
 [149 184]]
prediction: [[0.65894127 0

prediction: [[0.6876727  0.4584607 ]
 [0.63064176 0.4782473 ]
 [0.6588513  0.48444194]
 ...
 [0.44221342 0.34097123]
 [0.4424752  0.34204122]
 [0.44829005 0.34333488]]
75 times,test accuracy:0.549101
Precision 0.5538461538461539
Recall 0.10810810810810811
f1_score 0.18090452261306533
confusion_matrix
[[361  29]
 [297  36]]
prediction: [[0.4835278  0.32371935]
 [0.4041737  0.3363401 ]
 [0.4419677  0.32971   ]
 ...
 [0.2942642  0.33904085]
 [0.29347408 0.33909658]
 [0.29905382 0.3393661 ]]
76 times,test accuracy:0.560166
Precision 0.5418994413407822
Recall 0.2912912912912913
f1_score 0.37890625000000006
confusion_matrix
[[308  82]
 [236  97]]
prediction: [[0.67633456 0.49232107]
 [0.6147058  0.5096798 ]
 [0.63940287 0.51074284]
 ...
 [0.4349136  0.40923405]
 [0.4348406  0.4102755 ]
 [0.44068626 0.41125792]]
77 times,test accuracy:0.5504841
Precision 0.5307692307692308
Recall 0.2072072072072072
f1_score 0.2980561555075594
confusion_matrix
[[329  61]
 [264  69]]
prediction: [[0.27366003 0.

In [3]:
import tensorflow as tf
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from datetime import datetime

import sklearn
print("start")

f=open(r'.\交通银行601328.csv')
d=pd.read_csv(f)

d= d.sort_index(ascending=False)  #按照index降序排列
#print(d)
#print("///////////////////////////////")
d=d.reset_index(drop=True)
#print(d)
d_row=d.shape[0]
d['label']=1  #我们认为1代表涨，0代表不涨不跌，-1代表跌
#print(d)

#print(d.iloc[1,0])
#print(d.iloc[1,2])
#print(d.iloc[0,2])
#  -------------------建立标签----------------
for i in range(1,d_row):   
    if(d.iloc[i,2]>d.iloc[i-1,2]):
        d.iloc[i,6]=1
    elif(d.iloc[i,2]==d.iloc[i-1,2]):
        d.iloc[i,6]=0
    else:
        d.iloc[i,6]=-1

d=d.drop([0])
d=d.reset_index(drop=True)
d_row=d.shape[0]  #删除第一行以后，现在是2418行了

#print(d)
d=d['change']
print(d)



start


KeyError: 'change'

In [4]:
import tensorflow as tf
import numpy as np
import pandas as pd
import sklearn as sk

import matplotlib.pyplot as plt
from datetime import datetime

import sklearn
print("start")

y_true=[1]*5
y_pred=[0]*5
y_pred[0]=1
y_pred[4]=1

print(y_true)
print(y_pred)

print("Precision", sk.metrics.precision_score(y_true, y_pred))
print( "Recall", sk.metrics.recall_score(y_true, y_pred))
print( "f1_score", sk.metrics.f1_score(y_true, y_pred))
print( "confusion_matrix")
print(sk.metrics.confusion_matrix(y_true, y_pred))


start
[1, 1, 1, 1, 1]
[1, 0, 0, 0, 1]
Precision 1.0
Recall 0.4
f1_score 0.5714285714285715
confusion_matrix
[[0 0]
 [3 2]]
